In [1]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np

In [2]:
url_front = 'http://data.insideairbnb.com/singapore/sg/singapore/'
url_end = '/data/listings.csv.gz'

# Clean Listings

In [3]:
dates = ['2019-03-18', '2019-04-20', '2019-05-23', '2019-06-25', '2019-07-21', '2019-08-28', '2019-09-25', '2019-10-25', '2019-11-26', '2019-12-28',
         '2020-01-26', '2020-02-27', '2020-03-21', '2020-04-26', '2020-05-27', '2020-06-22', '2020-10-26', '2020-11-27', '2020-12-29','2021-01-27']

In [4]:
listings = []

In [5]:
for date in dates:
    # print(date)
    listing = pd.read_csv(url_front+date+url_end)
    listings.append(listing)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (42,43,61,62,94,95) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
# 2020-10 onwards some features were lost.
long_features = listings[0].columns.values
short_features = listings[-1].columns.values
keep_features = [x for x in long_features if x in short_features]

In [7]:
keep_features

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'name',
 'description',
 'neighborhood_overview',
 'picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'amenities',
 'price',
 'minimum_nights',
 'maximum_nights',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'minimum_maximum_nights',
 'maximum_maximum_nights',
 'minimum_nights_avg_ntm',
 'maximum_nights_avg_ntm',
 'calendar_updated',
 'has_availability',
 'availability_30',
 'availability_60',
 'avail

In [8]:
listings_full = listings[0][keep_features]
for i in range(1, len(dates) - 1):
    listings_full = pd.concat([listings_full, listings[i][keep_features]])

In [9]:
delete_col = ['listing_url',
 'scrape_id',
 'last_scraped',
 'picture_url',
 'host_url',
 'host_name',
 'host_location',
 'host_acceptance_rate',
 'host_thumbnail_url',
 'host_picture_url',
 'host_listings_count',
 'host_total_listings_count',
 'neighbourhood',
 'has_availability',
 'calendar_last_scraped',
 'first_review',
 'last_review',
 'license',
 'calculated_host_listings_count_entire_homes',
 'calculated_host_listings_count_private_rooms',
 'calculated_host_listings_count_shared_rooms']

In [10]:
listings_cleaned = listings_full.drop(columns = delete_col)

In [11]:
listings_cleaned = listings_cleaned.drop_duplicates()
listings_cleaned = listings_cleaned.reset_index().drop(columns = ['index'])

In [12]:
listings_cleaned.shape

(138263, 51)

# Find Nearby Sites and Stations

In [13]:
# Extract the location data.
df_lat_lon = listings_cleaned[['latitude','longitude']].reset_index(drop=True)

In [15]:
# Read in the locations of popular sites
df_site = pd.read_csv('../../Datasets/popular_sites.csv').drop(['Image'],axis=1)
df_site.head()

,Title,kfi5wf,jdzyld,nfofm
0,Marina Bay Sands Singapore,4.5,"(7,026)",High-end hotel with bay views & dining
1,Gardens by the Bay,4.7,"(107,063)",Modern public gardens on reclaimed land
2,Sentosa,4.6,"(3,415)",Palawan Beach & Tiger Sky Tower
3,Merlion,4.6,"(25,932)","Half-lion, half-fish statue & fountain"
4,Universal Studios Singapore,4.6,"(77,742)",Movie- & TV-themed amusement park


In [16]:
# Read in the locations of nearby MRT Stations
df_mrt = pd.read_csv('../../Datasets/sg_mrt.csv')
df_mrt.head()

,OBJECTID,STN_NAME,STN_NO,X,Y,Latitude,Longitude,COLOR
0,12,ADMIRALTY MRT STATION,NS10,24402.1063,46918.1131,1.440585,103.800998,RED
1,16,ALJUNIED MRT STATION,EW9,33518.6049,33190.0020,1.316433,103.882893,GREEN
2,33,ANG MO KIO MRT STATION,NS16,29807.2655,39105.7720,1.369933,103.849553,RED
3,81,BAKAU LRT STATION,SE3,36026.0821,41113.8766,1.388093,103.905418,OTHERS
4,80,BANGKIT LRT STATION,BP9,21248.2460,40220.9693,1.380018,103.772667,OTHERS


In [17]:
# Rename some attributes for easier recognization.
df_site.rename(columns={'Title':'site_name','nfofm':'intro','kfi5wf':'rating','jdzyld':'num_ratings'},inplace=True)
df_mrt.rename(columns={'STN_NAME':'station_name','STN_NO':'station_no','Latitude':'latitude','Longitude':'longitude','COLOR':'color'},inplace=True)

In [18]:
df_site.head()

,site_name,rating,num_ratings,intro
0,Marina Bay Sands Singapore,4.5,"(7,026)",High-end hotel with bay views & dining
1,Gardens by the Bay,4.7,"(107,063)",Modern public gardens on reclaimed land
2,Sentosa,4.6,"(3,415)",Palawan Beach & Tiger Sky Tower
3,Merlion,4.6,"(25,932)","Half-lion, half-fish statue & fountain"
4,Universal Studios Singapore,4.6,"(77,742)",Movie- & TV-themed amusement park


In [19]:
df_site.num_ratings = df_site.num_ratings.dropna().apply(lambda x: int(x[2:-1].replace(',','')))

In [20]:
df_site.num_ratings.describe()

count        98.000000
mean       8420.408163
std       16198.456969
min           1.000000
25%         895.500000
50%        3270.500000
75%        7139.000000
max      107063.000000
Name: num_ratings, dtype: float64

In [21]:
# Rate popularity using Bayesian Probability.
# Choose the average value of ratings R and total expected number of ratings W.
R = df_site[df_site.num_ratings >= 1000].rating.median() # choose only ratings with 1000+ number of ratings.
W = df_site.num_ratings.mean() / 5 

df_site['norm_rating'] = (df_site.rating * df_site.num_ratings + R * W) / (df_site.num_ratings + W)
df_site.norm_rating.fillna(4.5) # The missing data is at Clark Quey whose rating was missing and then imputed manually.

0     4.500000
1     4.696903
2     4.566973
3     4.593902
4     4.597880
        ...   
94    4.500000
95    4.360272
96    4.427001
97    4.368015
98    4.465705
Name: norm_rating, Length: 99, dtype: float64

In [22]:
df_site['norm_rating'].describe()

count    98.000000
mean      4.486507
std       0.110920
min       4.133485
25%       4.437528
50%       4.500000
75%       4.565143
max       4.696903
Name: norm_rating, dtype: float64

In [23]:
# Find the geolocation of these sites using external tools.
def find_geolocation(df,name):
    import json
    import requests

    longs = []
    lats = []
    for title in df[name]:
      url= 'https://developers.onemap.sg/commonapi/search?searchVal='+ '%20'.join(title.split()) +'&returnGeom=Y&getAddrDetails=Y&pageNum=1'
      response = requests.get(url)
      obj = response.json()
      if not obj['results']:
        longs.append('')
        lats.append('')
        continue
      longs.append(obj['results'][0]['LONGITUDE'])
      lats.append(obj['results'][0]['LATITUDE'])
    return longs, lats

In [24]:
longs,lats = find_geolocation(df_site,'site_name')
df_site['longitude'] = np.array(longs)
df_site['latitude'] = np.array(lats)

In [25]:
df_site = df_site[df_site.longitude != ''] # ignore unsuccessful queries.

In [26]:
# Function to find the distance between two coordinates.
from math import cos, asin, sqrt, pi

def distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742 * asin(sqrt(a)) #2*R*asin...

In [27]:
# Reusable functions for finding n(default=3) nearest sites of each listed hotel.
def find_nearby(df, df_nearby, name, thres=3):
  distances = []
  for i in range(df.shape[0]):
    lat1 = float(df.latitude[i])
    lon1 = float(df.longitude[i])
    candidates = []
    for j in range(df_nearby.shape[0]):
        lat2 = float(df_nearby.latitude.iloc[j])
        lon2 = float(df_nearby.longitude.iloc[j])
        dist = distance(lat1, lon1, lat2, lon2)
        candidates.append([df_nearby[name].iloc[j],dist])
    candidates.sort(key=lambda x:x[1])
    distances.append(candidates[:thres])
  return distances

In [28]:
# Weighted ratings for the 3 nearby locations.
def avg_rate(x,thres,df,feature_n,rating_n):
    from math import sqrt
    
    sum_dist = 0
    sum_rating = 0
    for i in range(thres):
        sum_dist += x[i][1]
        sum_rating += df.loc[df[feature_n] == x[i][0], rating_n].values[0]
    return sqrt(sum_rating/sum_dist)

In [29]:
distances_site = find_nearby(df_lat_lon, df_site, 'site_name')
listings_cleaned['nearby_sites_rating'] = np.array([avg_rate(x,3,df_site,'site_name','norm_rating') 
                                 for x in distances_site])

In [30]:
distances_mrt = find_nearby(df_lat_lon, df_mrt,'station_name',1)
listings_cleaned['nearest_station_dist'] = np.array([x[0][1] for x in distances_mrt])

In [31]:
# Include the distance to the nearest popular site as well as its rating.
listings_cleaned['nearest_site_dist'] = np.array([x[0][1] for x in distances_site])
listings_cleaned['nearest_site_rating'] = np.array([df_site[df_site.site_name == x[0][0]].norm_rating.values[0] for x in distances_site])

In [32]:
listings_cleaned.head()

,id,name,description,neighborhood_overview,host_id,host_since,host_about,host_response_time,host_response_rate,host_is_superhost,...,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,reviews_per_month,nearby_sites_rating,nearest_station_dist,nearest_site_dist,nearest_site_rating
0,49091,COZICOMFORT LONG TERM STAY ROOM 2,This is Room No. 2.(available after Jan 15 201...,NaN,266763,2010-10-20,I am a private tutor by profession. My husband...,within a few hours,100%,f,...,10.0,8.0,8.0,f,2,0.02,0.995731,0.617762,4.295315,4.595264
1,50646,Pleasant Room along Bukit Timah,Fully furnished bedroom with a nice view on th...,The serenity & quiet surrounding makes it an i...,227796,2010-09-08,"I am a working professional, living in Singapo...",NaN,NaN,f,...,10.0,9.0,9.0,f,1,0.30,1.269606,0.399696,2.039236,4.654423
2,56334,COZICOMFORT,This is Room no. 1. USD700 per month On the fa...,NaN,266763,2010-10-20,I am a private tutor by profession. My husband...,within a few hours,100%,f,...,10.0,8.0,9.0,f,2,0.21,0.995281,0.524353,4.290337,4.595264
3,71609,Ensuite Room (Room 1 & 2) near EXPO,For 3 (Website hidden by Airbnb) room 1&2 and ...,NaN,367042,2011-01-29,Hi My name is Belinda -Housekeeper \r\n\r\nI w...,within an hour,100%,f,...,9.0,8.0,8.0,t,9,0.14,1.116033,0.486745,2.648653,4.456579
4,71896,B&B Room 1 near Airport & EXPO,Vocational Stay Deluxe Bedroom in Singapore.(N...,NaN,367042,2011-01-29,Hi My name is Belinda -Housekeeper \r\n\r\nI w...,within an hour,100%,f,...,9.0,8.0,8.0,t,9,0.22,1.113617,0.481193,2.436100,4.456579


In [33]:
listings_cleaned.to_csv('../../Datasets/listings_cleaned.csv',index=False)